In [ ]:
# !pip install vaderSentiment
# !pip install tabulate
# !pip install textblob

In [ ]:
import warnings
import textblob

import numpy as np
import pandas as pd
import matplotlib.pyplot as plt

from utils import *
from doc_utils import *
from tabulate import tabulate
from collections import Counter
from sklearn.ensemble import ExtraTreesClassifier
from nlp_utils import get_features, make_predictions
from sklearn.exceptions import UndefinedMetricWarning
from sklearn.metrics import precision_score, recall_score, f1_score
from vaderSentiment.vaderSentiment import SentimentIntensityAnalyzer

In [ ]:
plt.rcParams['figure.figsize'] = [20, 8]

# Read tweets csv
df = pd.concat([pd.read_csv("../data/Tweets3WeeksLocations.csv"), pd.read_pickle("../data/Tweets3Weeks_part2.pkl")]) 
# Drop all-null rows if any
df.dropna(how='all', inplace=True)
df.drop(columns=['Unnamed: 0'], inplace=True)

In [ ]:
lead_days = 2
days_window = 5

start_date = pd.to_datetime("23-feb-2019")
end_date = pd.to_datetime("15-apr-2019")


labels = process_acled_csv("../data/1900-01-01-2019-04-15-India.csv", 
                           top_locations=-1, 
                           start=start_date, 
                           end=end_date,
                           lead_days=lead_days,
                           days_window = days_window)

In [ ]:
clean = False
if not clean:
    df = clean_df(df)
    clean = True

In [ ]:
df['entities'] = df['entities'].apply(ast.literal_eval)

In [ ]:
import ast

In [ ]:
ast.literal_eval(df['entities'].iloc[0])

In [ ]:
print(json.loads(df['entities'].iloc[0]))

In [ ]:
df['entities'] = df['entities'].apply(lambda x: )

In [ ]:
type(df['entities'].iloc[0])

In [ ]:
df.head(5)

In [ ]:
df = get_tweet_sentiment(df)

In [ ]:
df.columns

In [ ]:
df['neu']

In [ ]:
plot_counter(df['lang'], num_elements=10, xlabel="Language", ylabel="Number of Tweets", title="Language Distribution")

In [ ]:
df['created_at'] = pd.to_datetime(df['created_at'])
# Sort by time created
df.sort_values(by=['created_at'], inplace=True)

In [ ]:
print(start_date, end_date)

warnings.filterwarnings('ignore')
location_date_dict = interleave_location_and_date(df, start_date, end_date)
warnings.filterwarnings('default')

### Current Features
   * Number of tweets each day
   * Average pos, neg, neu and compound features
   * Tweet count with neg sentiment

### TODO
   * Hate speech
   * Violent speech

In [ ]:
location_features_dict = {}
for location in location_date_dict:
    location_features_dict[location] = get_features(location_date_dict[location])
    print(location, "done")

In [ ]:
make_predictions(location_features_dict, labels)